In [195]:
import numpy as np
import pandas as pd
from collections import defaultdict
from matplotlib import pyplot as plt
from itertools import permutations

In [196]:
import warnings
warnings.filterwarnings("ignore")

In [197]:
# the goal is to now build the reward function for each state-action mapping
# two actiuons: trade or not trade
# if the action is to not trade, the reward is simply the average metric you observe (e.g. the performance of the player for a certain tenure + tier)
# if the action is to trade, the reward is fixed: the average performance of the player after the trade

In [198]:
SAMPLE = False
if SAMPLE:
    # generate the filenames for basketball positions
    filenames = ["data/sample_aggregate/C_sample_aggregate.csv", 
                 "data/sample_aggregate/PF_sample_aggregate.csv", 
                 "data/sample_aggregate/PG_sample_aggregate.csv",
                 "data/sample_aggregate/SF_sample_aggregate.csv",
                 "data/sample_aggregate/SG_sample_aggregate.csv"]
else:
    filenames = ["data/aggregate/C_aggregate.csv",
                 "data/aggregate/PF_aggregate.csv",
                 "data/aggregate/PG_aggregate.csv",
                 "data/aggregate/SF_aggregate.csv",
                 "data/aggregate/SG_aggregate.csv"]

In [199]:
positions = ["C", "PF", "PG", "SF", "SG"]
position_dfs = defaultdict(pd.DataFrame)
for i, filename in enumerate(filenames):
    position_dfs[positions[i]] = pd.read_csv(filename, index_col=0)

In [200]:
pos_bins = defaultdict(list)
for pos in positions:
    position_dfs[pos]["METRIC"] = position_dfs[pos]["METRIC"].astype(float)
    position_dfs[pos]["TIER"], bins = pd.qcut(position_dfs[pos]["METRIC"], 4, labels=["D", "C", "B", "A"], retbins=True)
    pos_bins[pos] = bins

position_dfs["C"].head()

,PLAYER_NAME,PLAYER_ID,TEAM_NAME,TEAM_ABBREVIATION,GAME_DATE,MATCHUP,METRIC,TIER
0,Al Horford,201143,Atlanta Hawks,ATL,2013-10-30T00:00:00,ATL @ DAL,1.636125,C
1,Al Horford,201143,Atlanta Hawks,ATL,2013-11-01T00:00:00,ATL vs. TOR,2.023265,B
2,Al Horford,201143,Atlanta Hawks,ATL,2013-11-03T00:00:00,ATL @ LAL,1.601184,C
3,Al Horford,201143,Atlanta Hawks,ATL,2013-11-05T00:00:00,ATL @ SAC,3.181728,A
4,Al Horford,201143,Atlanta Hawks,ATL,2013-11-07T00:00:00,ATL @ DEN,2.387248,A


In [201]:
# the trade reward needs to take into account the salary of the player (scaled)
# if the action is not to trade, the salary of the player is a negative cost on their metric (and thus accumulates over tenure)
# if the action is to trade, the salary does not impact the reward (the player is no longer on the team)

In [202]:
def get_player_tiers():
    indi_playertiers = []
    for pos in positions:
        pos_df = position_dfs[pos]
        pos_df.loc[:, ["PLAYER_NAME", "TIER"]].groupby("TIER").count()
        # get the max count TIER for each PLAYER_NAME
        indi_playertiers.append(pos_df.loc[:, ["PLAYER_NAME", "TIER"]].groupby("PLAYER_NAME").agg(lambda x: x.value_counts().index[0]))
    return pd.concat(indi_playertiers)


In [203]:
def get_avg_salary_by_pos_tier():
    salary = pd.read_csv("data/avg_player_salary.csv", index_col=0)

    all_salary_tiers = salary.merge(get_player_tiers().reset_index(), left_on="name", right_on="PLAYER_NAME")
    return all_salary_tiers.loc[:, ["TIER", "position", "annual_salary"]].groupby(["position", "TIER"]).mean()

In [204]:
# calculate average metric per tenure-tier combination for every position
def generate_notrade_reward(pos):
    pos_df = position_dfs[pos]

    pos_df['TENURE'] = (pos_df[['PLAYER_NAME', 'TEAM_NAME']] != pos_df[['PLAYER_NAME', 'TEAM_NAME']].shift()).any(axis=1).cumsum() - 1

    # Reset 'TENURE' to start from 0 at each team or player change
    changes = (pos_df[['PLAYER_NAME', 'TEAM_NAME']] != pos_df[['PLAYER_NAME', 'TEAM_NAME']].shift()).any(axis=1)
    pos_df['TENURE'] = pos_df.groupby(changes.cumsum())['TENURE'].cumcount()

    avg_metric_by_tier = pos_df.loc[:, ["TIER", "METRIC"]].groupby(["TIER"]).mean().reset_index()
    avg_metric_by_tier_tenure = pos_df.loc[:, ["TENURE", "TIER", "METRIC"]].groupby(["TENURE", "TIER"]).mean().reset_index()

    tenures = np.arange(0, 1000)
    tiers = ["A", "B", "C", "D"]
    avg_performance = pd.MultiIndex.from_product([tenures, tiers], names=["TENURE", "TIER"])
    avg_metric = np.mean(avg_metric_by_tier.values[:, 1])
    avg_performance = pd.DataFrame(avg_metric, index=avg_performance, columns=["METRIC"])
    avg_performance = avg_performance.reset_index()

    reward_notrade = avg_metric_by_tier_tenure.merge(avg_metric_by_tier, on="TIER", how="left")
    reward_notrade['METRIC'] = reward_notrade['METRIC_x'].combine_first(reward_notrade['METRIC_y'])
    reward_notrade = reward_notrade.drop(columns=['METRIC_x', 'METRIC_y'])

    reward_notrade = reward_notrade.merge(avg_performance, on=["TENURE", "TIER"], how="outer")
    reward_notrade = reward_notrade.sort_values(by=["TENURE", "TIER"], ascending=True)
    reward_notrade['METRIC'] = reward_notrade['METRIC_x'].combine_first(reward_notrade['METRIC_y'])
    reward_notrade = reward_notrade.drop(columns=['METRIC_x', 'METRIC_y'])
    return reward_notrade

In [205]:
# generate rewards for each position if action is to not trade, adjust for salary, and save to csv

for pos in positions:
    reward_notrade = generate_notrade_reward(pos)
    salary_info = get_avg_salary_by_pos_tier().reset_index()
    salary_info = salary_info.loc[salary_info["position"] == pos, :].loc[:, ["TIER", "annual_salary"]]
    adj_metric_by_tier = reward_notrade.merge(salary_info)
    SALARY_SCALING = (10)**(-8) # from tens of millions of dollars to a scale of 0-10
    adj_metric_by_tier["ADJ_METRIC"] = adj_metric_by_tier["METRIC"] - SALARY_SCALING*adj_metric_by_tier["annual_salary"]
    adj_metric_by_tier_out = adj_metric_by_tier.sort_values(["TENURE", "TIER"]).reset_index(drop=True).loc[:, ["TENURE", "TIER", "ADJ_METRIC"]]
    adj_metric_by_tier_out.to_csv("rewards/team/reward_notrade_{}.csv".format(pos))

In [206]:
# now to build the reward function for trading
# the reward is the average performance of the player after the trade

In [207]:
for pos in positions:
    pos_df = position_dfs[pos]

    pos_df["LAG_TEAM"] = pos_df["TEAM_NAME"].shift(1)
    pos_df = pos_df.dropna()
    traded = pos_df.loc[pos_df["TEAM_NAME"] != pos_df["LAG_TEAM"], ["TIER", "METRIC"]]
    trade_reward = traded.groupby(["TIER"]).mean().reset_index()
    trade_reward["METRIC"] = -1*trade_reward["METRIC"] #this is a cost (they no longer have this player)
    trade_reward_out = trade_reward.sort_values(["TIER"], ascending=False).reset_index(drop=True)
    trade_reward_out.to_csv("rewards/trade/reward_trade_{}.csv".format(pos))